In [3]:
import pandas as pd
import numpy as np

# Load the FIPS codes
dfDummy = pd.read_csv('./county_codes.csv')

fips_codes = dfDummy['FIPS'].values

# Generate random FIPS codes for 'OC_County_FIPS' and 'MR_County_FIPS'
mock_data = pd.DataFrame({
    'OC_County_FIPS': np.random.choice(fips_codes, size=100),
    'MR_County_FIPS': np.random.choice(fips_codes, size=100),
})

# Save the mock data to a CSV file
mock_data.to_csv('mock_data.csv', index=False)


In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
from tqdm import tqdm

def buildGraph(dataframe):
    nodes = dataframe[["OC_County_FIPS", "MR_County_FIPS"]].drop_duplicates().values.flatten()
    edges = list(zip(dataframe["OC_County_FIPS"], dataframe["MR_County_FIPS"]))

    G = nx.DiGraph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    return G

# SALSA algorithm (convergence criteria could be changed)
def salsa_algorithm(G, max_iter=100, tol=1.0e-6):
    nodes = G.nodes()
    num_nodes = len(nodes)

    hubs = {node: 1 for node in nodes}
    authorities = {node: 1 for node in nodes}

    for _ in tqdm(range(max_iter), desc="Running SALSA"):  # added tqdm here
        # Authority update step
        new_authorities = {}
        for node in nodes:
            new_authorities[node] = sum(
                hubs[neighbor] for neighbor in G.predecessors(node)
            )

        # Hub update step
        new_hubs = {}
        for node in nodes:
            new_hubs[node] = sum(
                authorities[neighbor] for neighbor in G.successors(node)
            )

        # Normalize
        auth_sum = sum(new_authorities.values())
        hub_sum = sum(new_hubs.values())

        for node in nodes:
            new_authorities[node] = new_authorities[node] / auth_sum
            new_hubs[node] = new_hubs[node] / hub_sum

        # Check for convergence
        err = sum(
            abs(new_authorities[node] - authorities[node]) for node in nodes
        ) + sum(abs(new_hubs[node] - hubs[node]) for node in nodes)
        if err < tol:
            break

        hubs, authorities = new_hubs, new_authorities

    return hubs, authorities

# The for loop to iterate through years will need to change to handle a dataframe input
def process_data_and_run_algorithms(dataframe):
    G = buildGraph(dataframe)
    salsa_hubs, salsa_authorities = salsa_algorithm(G)
    hits_hubs, hits_authorities = nx.hits(G, max_iter=100, tol=1.0e-6)
    nodes = list(G.nodes())
    salsa_hub_scores = [salsa_hubs[node] for node in nodes]
    salsa_authority_scores = [salsa_authorities[node] for node in nodes]
    hits_hub_scores = [hits_hubs[node] for node in nodes]
    hits_authority_scores = [hits_authorities[node] for node in nodes]

    dfScores = pd.DataFrame({
        "County": nodes,
        "SALSA_Hub_Score": salsa_hub_scores,
        "SALSA_Authority_Score": salsa_authority_scores,
        "HITS_Hub_Score": hits_hub_scores,
        "HITS_Authority_Score": hits_authority_scores,
    })
    return dfScores

# Example usage with a mock dataframe:
# df = pd.read_csv("path_to_your_mock_data.csv")
# scores = process_data_and_run_algorithms(df)
# scores.to_csv("year_scores.csv", index=False)
